<a href="https://colab.research.google.com/github/RimmalapudiRajesh/Assignment1/blob/main/Assignment_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import math

In [ ]:
x_order_rpt = pd.read_excel("/content/drive/MyDrive/Final Assignment Data/Company X - Order Report.xlsx")
x_pin_zone = pd.read_excel("/content/drive/MyDrive/Final Assignment Data/Company X - Pincode Zones.xlsx")
x_sku = pd.read_excel("/content/drive/MyDrive/Final Assignment Data/Company X - SKU Master.xlsx")
courier_invoice = pd.read_excel("/content/drive/MyDrive/Final Assignment Data/Courier Company - Invoice.xlsx")
courier_charges = pd.read_excel("/content/drive/MyDrive/Final Assignment Data/Courier Company - Rates.xlsx")

In [ ]:
a = pd.merge(x_order_rpt,x_sku,on ="SKU")
a["Total Weight as per X"] = ((a["Order Qty"]*a["Weight (g)"])/1000)

In [ ]:
b = pd.pivot_table(a,index = "ExternOrderNo",aggfunc = "sum")
b= b.reset_index()
b.drop(["Weight (g)","Order Qty"],axis =1, inplace = True)
b.columns = ["Order ID","Total Weight as per X (KG)"]
courier_invoice = pd.merge(courier_invoice,b, on="Order ID" )


<ipython-input-79-63e84ae14487>:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  b = pd.pivot_table(a,index = "ExternOrderNo",aggfunc = "sum")


In [ ]:

def calculate_weight_slab(weight):
    return ((math.ceil(weight/0.5))/2)

In [ ]:
courier_invoice["Weight_X"] = courier_invoice["Total Weight as per X (KG)"].apply(calculate_weight_slab)
courier_invoice["Weight slab charged by Courier Company (KG)"] = courier_invoice["Charged Weight"].apply(calculate_weight_slab)
courier_invoice.rename(columns = {'Zone':'Delivery Zone charged by Courier Company'}, inplace = True)
courier_invoice = pd.merge(courier_invoice,x_pin_zone,on ="Customer Pincode"	)

In [ ]:
def charges(Zone, Type_of_Shipment, Weight_X):

  if Type_of_Shipment == "Forward charges":
    if Zone  == "a":
        if Weight_X / 0.5 == 0.5:
          return 29.5
        else:
          a = Weight_X /0.5
          return (29.5 + (23.6*(a-1)))

    elif Zone  == "b":
        if Weight_X / 0.5 == 0.5:
          return 33.0
        else:
          a = Weight_X /0.5
          return (33+ (28.3*(a-1)))

    elif Zone  == "c":
        if Weight_X / 0.5 == 0.5:
          return 40.1
        else:
          a = Weight_X /0.5
          return (40.1+ (38.9*(a-1)))

    elif Zone  == "d":
        if Weight_X / 0.5 == 0.5:
          return 45.4
        else:
          a = Weight_X /0.5
          return (45.4+ (44.8*(a-1)))

    elif Zone  == "e":
        if Weight_X / 0.5 == 0.5:
          return 56.6
        else:
          a = Weight_X /0.5
          return (56.6+ (55.5*(a-1)))

  elif Type_of_Shipment == "Forward and RTO charges":
    if Zone  == "a":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 13.6)
        else:
          a = Weight_X /0.5
          return (29.5 + 13.6 + ((23.6*(a-1)*2)))

    elif Zone  == "b":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 20.5)
        else:
          a = Weight_X /0.5
          return (29.5 + 20.5 + ((23.6*(a-1)*2)))

    if Zone  == "c":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 31.9)
        else:
          a = Weight_X /0.5
          return (29.5 + 31.9 + ((23.6*(a-1)*2)))

    if Zone  == "d":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 41.3)
        else:
          a = Weight_X /0.5
          return (29.5 + 41.3 + ((23.6*(a-1)*2)))

    if Zone  == "e":
        if Weight_X / 0.5 == 0.5:
          return (29.5 + 55.5)
        else:
          a = Weight_X /0.5
          return (29.5 + 55.5 + ((23.6*(a-1)*2)))

In [ ]:
courier_invoice['Expected Charge as per X (Rs)'] = courier_invoice.apply(lambda row: charges(row['Zone'], row['Type of Shipment'], row['Weight_X']), axis=1)


In [ ]:
courier_invoice["Difference Between Expected Charges and Billed Charges (Rs.)"]=courier_invoice["Expected Charge as per X (Rs)"]-courier_invoice["Billing Amount (Rs.)"]
courier_invoice.rename(columns={"AWB Code":"AWB Number","Charged Weight":"Total weight as per Courier Company (KG)","Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.) " ,"Zone":"Delivery Zone as per X","Weight_X":"Weight slab as per X (KG)"},inplace=True)
courier_invoice.drop(columns=["Warehouse Pincode_y","Type of Shipment","Customer Pincode","Warehouse Pincode_x"],axis=1,inplace=True)

In [ ]:
courier_invoice = courier_invoice.drop_duplicates()

In [ ]:
courier_invoice.to_csv("Order_Level_Calculation.csv", index=False)


In [ ]:
courier_invoice

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Total Weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs),Difference Between Expected Charges and Billed Charges (Rs.)
0,1091117222124,2001806232,1.30,d,135.0,1.302,1.5,1.5,d,135.0,0.000000e+00
1,1091117222194,2001806273,1.00,d,90.2,0.615,1.0,1.0,d,90.2,-1.421085e-14
2,1091117222931,2001806408,2.50,d,224.6,2.265,2.5,2.5,d,224.6,0.000000e+00
3,1091117223244,2001806458,1.00,b,61.3,0.700,1.0,1.0,b,61.3,0.000000e+00
4,1091117229345,2001807012,0.15,d,45.4,0.240,0.5,0.5,d,45.4,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
167,1091117904860,2001811039,0.68,d,90.2,0.488,0.5,1.0,b,33.0,-5.720000e+01
169,1091121846136,2001811305,0.50,d,45.4,0.750,1.0,0.5,b,61.3,1.590000e+01
171,1091118551656,2001812941,0.73,d,90.2,0.500,0.5,1.0,b,33.0,-5.720000e+01
172,1091117614452,2001809383,0.50,d,86.7,0.607,1.0,0.5,b,97.2,1.050000e+01


In [ ]:
correctly_charged = courier_invoice[courier_invoice['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0]
overcharged = courier_invoice[courier_invoice['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0]
undercharged = courier_invoice[courier_invoice['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0]

correct_count = len(correctly_charged)
overcharge_count = len(overcharged)
undercharge_count = len(undercharged)

In [ ]:
correct_amount = correctly_charged['Expected Charge as per X (Rs)'].sum()
overcharge_amount = overcharged['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()
undercharge_amount = undercharged['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()



In [ ]:
dict_ = {"Metrics" : ["Total orders where X has been correctly charged", "Total Orders where X has been overcharged" , "Total Orders where X has been undercharged"],
         "Count" : [correct_count, overcharge_count, undercharge_count],
         "Amount" : [correct_amount,  overcharge_amount, undercharge_amount]}


summary_table = pd.DataFrame(dict_)
summary_table.to_csv("summary_table.csv", index=False)

In [ ]:
summary_table

,Metrics,Count,Amount
0,Total orders where X has been correctly charged,16,1268.6
1,Total Orders where X has been overcharged,96,-5143.0
2,Total Orders where X has been undercharged,12,547.5


In [ ]:
!pip install pandas openpyxl


In [ ]:
with pd.ExcelWriter('output_file.xlsx', engine='openpyxl') as writer:
    summary_table.to_excel(writer, sheet_name='SummaryTable', index=False)
    courier_invoice.to_excel(writer, sheet_name='OrderLevelCalculation', index=False)